# Oxygen Abundance

In [46]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

In [47]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.parser import parse

In [48]:
figure_setup()
settings = Settings()

In [49]:
N_BINS = 200
MIN_PART_COUNT = 1E0
MAX_PART_COUNT = 1E3

In [50]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [51]:
def read_data(simulation: str, tag_by_region: bool = False) -> pd.DataFrame:
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.add_metal_abundance(of="Fe", to='H')
    s.add_metal_abundance(of='O', to="Fe")

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "[Fe/H]": s.metal_abundance["Fe/H"][is_real_star & is_main_obj],
        "[O/Fe]": s.metal_abundance["O/Fe"][is_real_star & is_main_obj],}

    if tag_by_region:
        s.tag_particles_by_region(
            disc_std_circ=DISC_STD_CIRC,
            disc_min_circ=DISC_MIN_CIRC,
            cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
            bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)
        props["ComponentTag"] = s.region_tag[is_real_star & is_main_obj]

    return pd.DataFrame(props)

In [52]:
def plot_map_for_simulation(simulation: str) -> None:
    settings = Settings()
    galaxy = parse(simulation)[0]
    label = f"Au{galaxy}"
    df = read_data(simulation=simulation, tag_by_region=True)

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.set_xlim(-1, 1)
        ax.set_xticks([-0.5, 0, 0.5])
        ax.set_xlabel("[Fe/H]")

        ax.set_ylim(0.15, 0.35)
        ax.set_yticks([0.2, 0.25, 0.3])
        ax.set_ylabel("[O/Fe]")

        ax.tick_params(which='both', direction="in")
        ax.label_outer()

    for i in range(len(settings.components)):
        is_region = (df["ComponentTag"] == i)

        im = axs[i].hist2d(
            df["[Fe/H]"][is_region], df["[O/Fe]"][is_region],
            cmap='nipy_spectral', bins=N_BINS,
            range=[axs[i].get_xlim(), axs[i].get_ylim()],
            norm=mcolors.LogNorm(vmin=MIN_PART_COUNT, vmax=MAX_PART_COUNT),
            rasterized=True)[-1]

        axs[i].text(
            x=0.05, y=0.05, ha="left", va="bottom",
            s=r"$\textbf{" \
                + list(settings.component_labels.values())[i] + "}$",
            c=list(settings.component_colors.values())[i],
            size=8.0, transform=axs[i].transAxes)

    cbar = fig.colorbar(im, ax=axs[-1], orientation='vertical',
                        label=r'$N_\star$', pad=0)
    cbar.ax.set_yticks([1E0, 1E1, 1E2, 1E3])

    axs[0].text(x=axs[0].get_xlim()[0], y=axs[0].get_ylim()[1],
                s=r"$\texttt{" + label + "}$",
                size=8.0, ha='left', va='bottom',
                )

    fig.savefig(
        f"../images/metallicity/map_OFe_vs_FeH_{simulation}{SUFFIX}.pdf")
    plt.close(fig)

In [53]:
def plot_map_for_sample() -> None:
    fig = plt.figure(figsize=(7, 8))
    gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flatten():
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(-1, 1)
        ax.set_xticks([-0.5, 0, 0.5])
        ax.set_ylim(0.15, 0.35)
        ax.set_yticks([0.20, 0.25, 0.30])
        ax.set_axisbelow(True)
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel("[Fe/H]")
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel("[O/Fe]")

    for i in range(len(settings.groups["Included"])):
        ax = axs.flatten()[i]
        galaxy = settings.groups["Included"][i]
        simulation = f"au{galaxy}_or_l4_s127"
        label = f"Au{galaxy}"
        df = read_data(simulation=simulation)
        ax.hist2d(df["[Fe/H]"], df["[O/Fe]"],
                  cmap='nipy_spectral',
                  bins=N_BINS,
                  range=[ax.get_xlim(), ax.get_ylim()],
                  norm=mcolors.LogNorm(vmin=MIN_PART_COUNT,
                                       vmax=MAX_PART_COUNT),
                  rasterized=True)

        ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
                size=6.0, transform=ax.transAxes, ha='left', va='top')

        fig.savefig(
            f"../images/metallicity/map_OFe_vs_FeH_included{SUFFIX}.pdf")

    plt.close(fig)

In [54]:
plot_map_for_simulation(simulation="au6_or_l4_s127")

# SAMPLE = [f"au{i}_or_l4_s127" for i in settings.groups["Included"]]
# for simulation in SAMPLE:
#     plot_map_for_simulation(simulation=simulation)

# plot_map_for_sample()